In [215]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sko.GA import GA

# 读取四种材料的数据
data_material1 = pd.read_csv('data/材料1.csv')
data_material2 = pd.read_csv('data/材料2.csv')
data_material3 = pd.read_csv('data/材料3.csv')
data_material4 = pd.read_csv('data/材料4.csv')

# 添加材料标记
data_material1['磁芯材料'] = '材料1'
data_material2['磁芯材料'] = '材料2'
data_material3['磁芯材料'] = '材料3'
data_material4['磁芯材料'] = '材料4'

# 合并数据
data = pd.concat([data_material1, data_material2, data_material3, data_material4], ignore_index=True)


In [216]:
min(data['磁芯损耗，w/m3'])

415.6131139

In [217]:
# 磁通密度数据位于第5列到第1029列（索引为4到1028）
B_columns = data.columns[4:-1]

data['B_max'] = data[B_columns].max(axis=1)
data['B_min'] = data[B_columns].min(axis=1)

In [218]:
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler

# 选择特征和目标变量
X = data[['B_max', 'B_min', '励磁波形', '磁芯材料', '温度，oC', '频率，Hz']]
y = data['磁芯损耗，w/m3']

# 对类别特征进行独热编码
categorical_features = ['励磁波形', '磁芯材料']
X = pd.get_dummies(X, columns=categorical_features)

# 分离数值特征和类别特征
numerical_features = ['B_max', 'B_min', '温度，oC', '频率，Hz']
all_features = X.columns.tolist()  # 保存所有特征的名称

# 划分训练集和验证集
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

# 对数值特征进行标准化
scaler = StandardScaler()
X_train_numerical = scaler.fit_transform(X_train[numerical_features])
X_valid_numerical = scaler.transform(X_valid[numerical_features])

# 提取类别特征
X_train_categorical = X_train.drop(columns=numerical_features).values
X_valid_categorical = X_valid.drop(columns=numerical_features).values

# 合并数值特征和类别特征
X_train_processed = np.hstack((X_train_numerical, X_train_categorical))
X_valid_processed = np.hstack((X_valid_numerical, X_valid_categorical))

# 转换为 PyTorch 张量
X_train_tensor = torch.tensor(X_train_processed, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)

X_valid_tensor = torch.tensor(X_valid_processed, dtype=torch.float32)
y_valid_tensor = torch.tensor(y_valid.values, dtype=torch.float32)

# 创建数据集和数据加载器
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
valid_dataset = TensorDataset(X_valid_tensor, y_valid_tensor)

train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=512, shuffle=False)

In [219]:
# from sklearn.preprocessing import LabelEncoder

# # 对波形类型进行标签编码
# le_waveform = LabelEncoder()
# data['励磁波形编码'] = le_waveform.fit_transform(data['励磁波形'])

# # 对材料进行标签编码
# le_material_2 = LabelEncoder()
# data['材料编码'] = le_material_2.fit_transform(data['磁芯材料'])

In [220]:
# from sklearn.model_selection import train_test_split

# # 特征变量
# # X_other = data[['励磁波形编码', '材料编码', '温度，oC', '频率，Hz']]
# X_other = data[['温度，oC', '频率，Hz', '材料_材料1', '材料_材料2', '材料_材料3', '材料_材料4', '励磁波形_正弦波', '励磁波形_三角波', '励磁波形_梯形波']]
# # X = np.hstack((X_waveform, X_other))
# X = pd.concat([X_waveform, X_other], axis=1)
# print(X)
# # 目标变量：磁芯损耗
# y = data['磁芯损耗，w/m3']

# X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)


In [221]:
# import torch
# from torch import nn
# from torch.utils.data import DataLoader, TensorDataset
# from sklearn.preprocessing import StandardScaler

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train.values)
# X_valid = scaler.transform(X_valid.values)

# # 拆分训练集和验证集
# # X_all_train, X_all_valid, y_train, y_val = train_test_split(
# #     X_all, y, test_size=0.2, random_state=42)

# # 转换为 PyTorch 张量
# X_all_train_tensor = torch.tensor(X_train, dtype=torch.float32)
# y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)

# X_all_valid_tensor = torch.tensor(X_valid, dtype=torch.float32)
# y_val_tensor = torch.tensor(y_valid.values, dtype=torch.float32)

# train_dataset = TensorDataset(X_all_train_tensor, y_train_tensor)
# val_dataset = TensorDataset(X_all_valid_tensor, y_val_tensor)

# train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True)
# val_loader = DataLoader(val_dataset, batch_size=512, shuffle=False)

In [222]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class CoreLossCNNImproved(nn.Module):
    def __init__(self, input_size):
        super(CoreLossCNNImproved, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=16, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(input_size * 16, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 1)

    def forward(self, x):
        x = x.unsqueeze(1)  # 添加一个通道维度
        x = F.relu(self.conv1(x))
        x = x.view(x.size(0), -1)  # 展平
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        return x.squeeze()


# 实例化模型
input_size = X.shape[1]
model = CoreLossCNNImproved(input_size)

# 将模型移动到 GPU（如果可用）
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('mps')
model.to(device)

# model.load_state_dict(torch.load('cnn_model.pth'))

CoreLossCNNImproved(
  (conv1): Conv1d(1, 16, kernel_size=(3,), stride=(1,), padding=(1,))
  (fc1): Linear(in_features=176, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=1, bias=True)
)

In [223]:
import torch.optim as optim
import os

criterion = nn.MSELoss()
if os.path.exists('cnn_model_q5.pth'):
    model.load_state_dict(torch.load('cnn_model_q5.pth'))

else:

    optimizer = optim.Adam(model.parameters(), lr=0.01)

    num_epochs = 1000

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        running_mape = 0.0
        for x_all, labels in train_loader:
            x_all = x_all.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = model(x_all)
            loss = criterion(outputs, labels)
            mape = torch.mean(torch.abs(outputs - labels) / labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            running_mape += mape.item()
        avg_train_loss = running_loss / len(train_loader)
        avg_train_mape = running_mape / len(train_loader)

        # 验证
        model.eval()
        val_loss = 0.0
        val_mape = 0.0
        with torch.no_grad():
            for x_all, labels in valid_loader:
                x_all = x_all.to(device)
                labels = labels.to(device)

                outputs = model(x_all)
                loss = criterion(outputs, labels)
                mape = torch.mean(torch.abs(outputs - labels) / labels)
                val_loss += loss.item()
                val_mape += mape.item()
        avg_val_loss = val_loss / len(valid_loader)
        avg_val_mape = val_mape / len(valid_loader)
        if (epoch + 1) % 10 == 0:
            print(
                f'第 {epoch + 1} 轮训练，训练损失: {avg_train_loss / 1e6:.4f}，训练mape: {avg_train_mape:.4f}，验证损失: {avg_val_loss / 1e6:.4f}，验证mape: {avg_val_mape:.4f}, 验证mape: {avg_val_mape:.4f}')

    torch.save(model.state_dict(), 'cnn_model_q5.pth')

/var/folders/gy/6wf0tc3n7276h9vd9dqcpkvw0000gn/T/ipykernel_10985/2026451518.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('cnn_model_q

In [224]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, explained_variance_score, max_error, \
    mean_squared_log_error

with torch.no_grad():
    model.eval()
    y_cnn_pred = model(X_train_tensor.to(device)).cpu().numpy()
    mae = mean_absolute_error(y_train, y_cnn_pred)
    mse = mean_squared_error(y_train, y_cnn_pred)
    r2 = r2_score(y_train, y_cnn_pred)
    mape = np.mean(np.abs((y_train - y_cnn_pred) / y_train)) * 100
    rmse = np.sqrt(mse)
    median_ae = np.median(np.abs(y_train - y_cnn_pred))
    evs = explained_variance_score(y_train, y_cnn_pred)
    max_err = max_error(y_train, y_cnn_pred)
    # msle = mean_squared_log_error(y_train, y_cnn_pred)

    print(f'平均绝对误差（MAE）: {mae:.2f}\n')
    print(f'均方误差（MSE）: {mse:.2f}\n')
    print(f'均方根误差（RMSE）: {rmse:.2f}\n')
    print(f'决定系数（R2）: {r2:.2f}\n')
    print(f'平均绝对百分比误差（MAPE）: {mape:.2f}%\n')
    print(f'中位绝对误差（Median AE）: {median_ae:.2f}\n')
    print(f'解释方差分（EVS）: {evs:.2f}\n')
    print(f'最大误差（Max Error）: {max_err:.2f}\n')

平均绝对误差（MAE）: 7496.85

均方误差（MSE）: 198359089.41

均方根误差（RMSE）: 14084.00

决定系数（R2）: 1.00

平均绝对百分比误差（MAPE）: 15.43%

中位绝对误差（Median AE）: 3313.56

解释方差分（EVS）: 1.00

最大误差（Max Error）: 153963.41



In [225]:
# 定义温度的离散取值
T_values = np.array([25, 50, 70, 90])

# 定义可能的波形类型和材料类型
waveform_types = data['励磁波形'].unique()
material_types = data['磁芯材料'].unique()

# 保存原始类别特征的独热编码列名
waveform_columns = ['励磁波形_' + w for w in waveform_types]
material_columns = ['磁芯材料_' + m for m in material_types]
categorical_features = waveform_columns + material_columns


# 定义目标函数
def objective_function(v):
    # 提取变量
    B_max = v[0]
    B_min = v[1]
    waveform_idx = int(round(v[2]))  # 波形类型的索引
    material_idx = int(round(v[3]))  # 材料类型的索引
    T = v[4]  # 温度
    f = v[5]  # 频率

    # 将温度映射到最近的离散值
    T = T_values[np.argmin(np.abs(T_values - T))]
    # 确保索引在有效范围内
    waveform_idx = min(max(waveform_idx, 0), len(waveform_types) - 1)
    material_idx = min(max(material_idx, 0), len(material_types) - 1)

    # 获取波形和材料类型
    waveform = waveform_types[waveform_idx]
    material = material_types[material_idx]

    # 构建特征向量
    X_input = pd.DataFrame({
        'B_max': [B_max],
        'B_min': [B_min],
        '温度，oC': [T],
        '频率，Hz': [f]
    })

    # 创建独热编码的 DataFrame
    waveform_df = pd.DataFrame(0, index=[0], columns=waveform_columns)
    waveform_df['励磁波形_' + waveform] = 1
    material_df = pd.DataFrame(0, index=[0], columns=material_columns)
    material_df['磁芯材料_' + material] = 1

    # 合并所有特征
    X_input = pd.concat([X_input, waveform_df, material_df], axis=1)
    X_input = X_input[all_features]  # 确保列的顺序与训练时一致

    # 对数值特征进行标准化
    X_input_numerical = scaler.transform(X_input[numerical_features])

    # 取出类别特征
    X_input_categorical = X_input[categorical_features].values

    # 合并数值特征和类别特征
    X_input_processed = np.hstack((X_input_numerical, X_input_categorical))

    # 转换为张量
    X_input_tensor = torch.tensor(X_input_processed, dtype=torch.float32).to(device)

    # 预测磁芯损耗
    with torch.no_grad():
        P_pred = model(X_input_tensor).item()

    # 计算传输磁能
    E_trans = f * B_max

    # 定义目标函数，根据需求调整权重
    w1 = 1.0  # 磁芯损耗的权重
    w2 = 1.0  # 传输磁能的权重（根据量纲调整）
    obj_value = w1 * P_pred - w2 * E_trans  # 调整权重以平衡两个目标

    if P_pred == 0:
        return float('inf')

    return obj_value

In [226]:
# 定义变量的范围
var_bound = [
    (data['B_max'].min(), data['B_max'].max()),  # B_max
    (data['B_min'].min(), data['B_min'].max()),  # B_min
    (0, len(waveform_types) - 1 - 1e-6),  # 波形类型索引
    (0, len(material_types) - 1 - 1e-6),  # 材料类型索引
    (25, 90),  # 温度 T
    (50000, 500000)  # 频率 f
]

# 定义变量的精度
precision = [
    1e-4,  # B_max 精度
    1e-4,  # B_min 精度
    1,  # 波形类型索引精度
    1,  # 材料类型索引精度
    1,  # 温度精度
    1e3  # 频率精度
]

# 初始化遗传算法
ga = GA(
    func=objective_function,
    n_dim=len(var_bound),
    size_pop=50,
    max_iter=100,
    lb=[bound[0] for bound in var_bound],
    ub=[bound[1] for bound in var_bound],
    precision=precision,
)

# 运行优化
best_x, best_y = ga.run()

# 获取最佳参数
B_max_opt = best_x[0]
B_min_opt = best_x[1]
waveform_idx_opt = int(round(best_x[2]))
material_idx_opt = int(round(best_x[3]))
T_opt = T_values[np.argmin(np.abs(T_values - best_x[4]))]
f_opt = best_x[5]

waveform_opt = waveform_types[waveform_idx_opt]
material_opt = material_types[material_idx_opt]

print('最佳参数组合:')
print(f'B_max: {B_max_opt:.4f}')
print(f'B_min: {B_min_opt:.4f}')
print(f'励磁波形: {waveform_opt}')
print(f'磁芯材料: {material_opt}')
print(f'温度 T: {T_opt} ℃')
print(f'频率 f: {f_opt:.2f} Hz')
print(f'最优目标函数值: {best_y[0]:.4f}')

# 计算传输磁能和预测磁芯损耗
E_trans_opt = f_opt * B_max_opt

# 构建特征向量用于预测
X_input = pd.DataFrame({
    'B_max': [B_max_opt],
    'B_min': [B_min_opt],
    '温度，oC': [T_opt],
    '频率，Hz': [f_opt]
})

# 创建独热编码的 DataFrame
waveform_df = pd.DataFrame(0, index=[0], columns=waveform_columns)
waveform_df['励磁波形_' + waveform_opt] = 1
material_df = pd.DataFrame(0, index=[0], columns=material_columns)
material_df['磁芯材料_' + material_opt] = 1

# 合并所有特征
X_input = pd.concat([X_input, waveform_df, material_df], axis=1)
X_input = X_input[all_features]  # 确保列的顺序与训练时一致

# 对数值特征进行标准化
X_input_numerical = scaler.transform(X_input[numerical_features])

# 取出类别特征
X_input_categorical = X_input[categorical_features].values

# 合并数值特征和类别特征
X_input_processed = np.hstack((X_input_numerical, X_input_categorical))

# 转换为张量
X_input_tensor = torch.tensor(X_input_processed, dtype=torch.float32).to(device)

# 预测磁芯损耗
with torch.no_grad():
    P_pred = model(X_input_tensor).item()

print(f'预测磁芯损耗: {P_pred:.4f} W/m3')
print(f'传输磁能: {E_trans_opt:.4f} J')

最佳参数组合:
B_max: 0.1301
B_min: -0.1816
励磁波形: 正弦波
磁芯材料: 材料1
温度 T: 90 ℃
频率 f: 50172.00 Hz
最优目标函数值: -5719.8945
预测磁芯损耗: 805.4041 W/m3
传输磁能: 6525.2986 J


In [227]:
from pymoo.core.problem import ElementwiseProblem
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.optimize import minimize
# from pymoo.factory import get_termination

# 定义多目标问题类
class CoreLossParetoProblem(ElementwiseProblem):
    def __init__(self):
        super().__init__(n_var=len(var_bound),
                         n_obj=2,  # 两个目标
                         n_constr=0,  # 无约束
                         xl=[bound[0] for bound in var_bound],  # 下界
                         xu=[bound[1] for bound in var_bound])  # 上界

    def _evaluate(self, x, out, *args, **kwargs):
        B_max = x[0]
        B_min = x[1]
        waveform_idx = int(round(x[2]))
        material_idx = int(round(x[3]))
        T = x[4]
        f = x[5]
        
        # 计算传输磁能
        E_trans = f * B_max
        
        # 获取波形和材料
        waveform_opt = waveform_types[waveform_idx]
        material_opt = material_types[material_idx]

        # 预测磁芯损耗
        T_opt = T_values[np.argmin(np.abs(T_values - T))]
        X_input = pd.DataFrame({
            'B_max': [B_max],
            'B_min': [B_min],
            '温度，oC': [T_opt],
            '频率，Hz': [f]
        })

        waveform_df = pd.DataFrame(0, index=[0], columns=waveform_columns)
        waveform_df['励磁波形_' + waveform_opt] = 1
        material_df = pd.DataFrame(0, index=[0], columns=material_columns)
        material_df['磁芯材料_' + material_opt] = 1

        X_input = pd.concat([X_input, waveform_df, material_df], axis=1)
        X_input = X_input[all_features]

        X_input_numerical = scaler.transform(X_input[numerical_features])
        X_input_categorical = X_input[categorical_features].values

        X_input_processed = np.hstack((X_input_numerical, X_input_categorical))
        X_input_tensor = torch.tensor(X_input_processed, dtype=torch.float32).to(device)

        with torch.no_grad():
            P_pred = model(X_input_tensor).item()

        # 最小化磁芯损耗 (目标1) 和最大化传输磁能 (目标2, 负值，因为我们要最大化它)
        out["F"] = [P_pred, -E_trans]

# 初始化 NSGA2 算法
algorithm = NSGA2(pop_size=50)

# 定义优化问题
problem = CoreLossParetoProblem()

# 定义终止条件
# termination = get_termination("n_gen", 100)

# 运行多目标优化
res = minimize(
    problem,
    NSGA2(pop_size=50),
    termination=('n_gen', 100),  # 在这里指定终止条件
    seed=1,
    verbose=True
)

# 提取帕累托前沿解
pareto_front = res.F
pareto_solutions = res.X

# 输出帕累托前沿解
print("帕累托前沿解集：")
for i, (f1, f2) in enumerate(pareto_front):
    print(f"解 {i + 1}: 磁芯损耗 = {f1:.4f} W/m3, 传输磁能 = {-f2:.4f} J")
    B_max_opt, B_min_opt, waveform_idx_opt, material_idx_opt, T_opt, f_opt = pareto_solutions[i]
    waveform_opt = waveform_types[int(round(waveform_idx_opt))]
    material_opt = material_types[int(round(material_idx_opt))]
    
    print(f'B_max: {B_max_opt:.4f}, B_min: {B_min_opt:.4f}, 励磁波形: {waveform_opt}, 材料: {material_opt}, 温度: {T_opt} ℃, 频率: {f_opt:.2f} Hz\n')

# 从帕累托前沿解集中选择某个最优解进行进一步计算
best_solution_idx = 0  # 可以选择第一个帕累托解
B_max_opt, B_min_opt, waveform_idx_opt, material_idx_opt, T_opt, f_opt = pareto_solutions[best_solution_idx]

waveform_opt = waveform_types[int(round(waveform_idx_opt))]
material_opt = material_types[int(round(material_idx_opt))]

print("最优参数如下：")
print(f"\t 温度: {T_opt}")
print(f"\t 频率: {f_opt}")
print(f"\t 波形: {waveform_idx_opt}")
print(f"\t 材料: {material_idx_opt}")
print(f"\t B_max: {B_max_opt}")
print(f"\t B_min: {B_min_opt}")
# 计算传输磁能和预测磁芯损耗
E_trans_opt = f_opt * B_max_opt

# 构建特征向量用于预测
X_input = pd.DataFrame({
    'B_max': [B_max_opt],
    'B_min': [B_min_opt],
    '温度，oC': [T_opt],
    '频率，Hz': [f_opt]
})

# 创建独热编码的 DataFrame
waveform_df = pd.DataFrame(0, index=[0], columns=waveform_columns)
waveform_df['励磁波形_' + waveform_opt] = 1
material_df = pd.DataFrame(0, index=[0], columns=material_columns)
material_df['磁芯材料_' + material_opt] = 1

# 合并所有特征
X_input = pd.concat([X_input, waveform_df, material_df], axis=1)
X_input = X_input[all_features]  # 确保列的顺序与训练时一致

# 对数值特征进行标准化
X_input_numerical = scaler.transform(X_input[numerical_features])

# 取出类别特征
X_input_categorical = X_input[categorical_features].values

# 合并数值特征和类别特征
X_input_processed = np.hstack((X_input_numerical, X_input_categorical))

# 转换为张量
X_input_tensor = torch.tensor(X_input_processed, dtype=torch.float32).to(device)

# 预测磁芯损耗
with torch.no_grad():
    P_pred = model(X_input_tensor).item()

print(f'预测磁芯损耗: {P_pred:.4f} W/m3')
print(f'传输磁能: {E_trans_opt:.4f} J')

n_gen  |  n_eval  | n_nds  |      eps      |   indicator  
     1 |       50 |     11 |             - |             -
     2 |      100 |     12 |  0.0329688042 |         ideal
     3 |      150 |     20 |  0.0179547204 |         ideal
     4 |      200 |     28 |  0.0159304959 |         ideal
     5 |      250 |     24 |  0.0082583704 |         nadir
     6 |      300 |     25 |  0.0280609411 |         ideal
     7 |      350 |     31 |  0.0039051249 |         ideal
     8 |      400 |     34 |  0.0059632412 |         ideal
     9 |      450 |     43 |  0.0160069154 |         ideal
    10 |      500 |     47 |  0.0045523790 |         nadir
    11 |      550 |     50 |  0.0060495669 |             f
    12 |      600 |     50 |  0.0145837335 |         ideal
    13 |      650 |     50 |  0.0294119737 |         ideal
    14 |      700 |     50 |  0.0401598023 |         nadir
    15 |      750 |     50 |  0.0050584782 |             f
    16 |      800 |     50 |  0.0053749525 |         ide

最佳参数组合:

温度 T: 82.772 ℃

频率 f: 331546 Hz

磁通密度峰值 B_max: 0.0971 T

磁通密度峰值 B_min: -0.1531 T

波形类型: 正弦波

磁芯材料: 材料1

预测磁芯损耗: 151.4120 W/m3

传输磁能: 32219.1351 J
